In [2]:
import pandas as pd
import csv
import re

df = pd.read_csv('generate_cypher_4bit_error_evaluated.csv')
pd.set_option('display.max_colwidth', None)
pd.set_option("display.max_rows", None)

In [3]:
no_err = df[(df["error_analysis"] == "NoError")]
err = df[(df["error_analysis"] != "NoError")]
executable_df = no_err[no_err["exact_match"] == 0.0]
matching_df = no_err[no_err["exact_match"] == 1.0]

In [4]:
avg_len_executable = executable_df["question"].str.len().mean()
avg_len_matching = matching_df["question"].str.len().mean()
avg_len_err = err['question'].str.len().mean()

avg_len_executable, avg_len_matching, avg_len_err

(75.00819061801936, 70.67391304347827, 83.56299212598425)

In [9]:
len(matching_df), len(executable_df), len(no_err), len(df)

(874, 1343, 2217, 2471)

In [246]:
# no_err_counts = no_err["database_reference_alias"].value_counts()
err_counts = err["database_reference_alias"].value_counts()
executable_counts = executable_df["database_reference_alias"].value_counts()
matching_counts = matching_df["database_reference_alias"].value_counts()

# no_err_table = no_err_counts.reset_index().rename(columns={"index": "database_reference_alias", "database_reference_alias": "count"})
err_table = err_counts.reset_index().rename(columns={"index": "database_reference_alias", "database_reference_alias": "count"})
executable_table = executable_counts.reset_index().rename(columns={"index": "database_reference_alias", "database_reference_alias": "count"})
matching_table = matching_counts.reset_index().rename(columns={"index": "database_reference_alias", "database_reference_alias": "count"})

# print("No Error Table:\n", no_err_table)
print("\nError Table:\n", err_table)
print("\nNot Matching Table:\n", executable_table)
print("\nMatching Table:\n", matching_table)





Error Table:
              database_reference_alias  count
0   neo4jlabs_demo_db_recommendations     37
1         neo4jlabs_demo_db_northwind     27
2            neo4jlabs_demo_db_movies     26
3         neo4jlabs_demo_db_companies     25
4            neo4jlabs_demo_db_fincen     20
5            neo4jlabs_demo_db_twitch     19
6           neo4jlabs_demo_db_neoflix     18
7        neo4jlabs_demo_db_grandstack     16
8           neo4jlabs_demo_db_network     14
9      neo4jlabs_demo_db_buzzoverflow     12
10   neo4jlabs_demo_db_stackoverflow2     12
11    neo4jlabs_demo_db_gameofthrones      9
12    neo4jlabs_demo_db_offshoreleaks      7
13          neo4jlabs_demo_db_twitter      7
14          neo4jlabs_demo_db_bluesky      4
15    neo4jlabs_demo_db_openstreetmap      1

Not Matching Table:
              database_reference_alias  count
0         neo4jlabs_demo_db_companies    171
1   neo4jlabs_demo_db_recommendations    144
2           neo4jlabs_demo_db_neoflix    140
3           neo4jl

In [11]:
executable_df[executable_df["bleu_score"] < 0.2].sort_values(by='bleu_score', ascending=True)[["question","cypher","generated_cypher"]]

,question,cypher,generated_cypher
2168,"Find the streams that have ""rajjix"" as a chatter and play ""Dota 2"".","MATCH (u:User {name: ""rajjix""}) MATCH (u)-[:CHATTER]->(s:Stream) MATCH (s)-[:PLAYS]->(g:Game {name: ""Dota 2""}) RETURN s.name","MATCH (s:Stream)-[:CHATTER]->(u:User {name: 'rajjix'}) WHERE s.name IS NOT NULL AND s.name <> '' AND s.name <> 'rajjix' RETURN s.name, s.url, s.total_view_count, s.followers, s.createdAt, s.description, s.id, s.name AS StreamName, s.url AS StreamUrl, s.total_view_count AS StreamTotalViewCount, s.followers AS StreamFollowers, s.createdAt AS StreamCreatedAt, s.description AS StreamDescription, s.id AS StreamId, s.name AS StreamStreamName, s.url AS StreamStreamUrl, s.total_view_count AS StreamStreamTotalViewCount, s.followers AS StreamStreamFollowers, s.createdAt AS StreamStreamCreatedAt, s.description AS StreamStreamDescription, s.id AS StreamStreamId, s.name AS StreamStreamStreamName, s.url AS StreamStreamStreamUrl, s.total_view_count AS StreamStreamStreamTotalViewCount, s.followers AS StreamStreamStreamFollowers, s.createdAt AS StreamStreamStreamCreatedAt, s.description AS StreamStreamStreamDescription, s.id AS StreamStreamStreamId, s.name AS StreamStreamStreamStreamName, s.url AS StreamStreamStreamStreamUrl, s.total_view_count AS StreamStreamStreamStreamTotalViewCount, s.followers AS StreamStreamStreamStreamFollowers, s.createdAt AS StreamStreamStreamStreamCreatedAt, s.description AS StreamStreamStreamStreamDescription, s.id AS StreamStreamStreamStreamId, s.name AS StreamStreamStreamStreamStreamName, s.url AS StreamStreamStreamStreamStreamUrl, s.total_view_count AS StreamStreamStreamStreamStreamTotalViewCount, s.followers AS StreamStreamStreamStreamStreamFollowers, s.createdAt AS StreamStreamStreamStreamStreamCreatedAt, s.description AS StreamStreamStreamStreamStreamDescription, s.id AS StreamStreamStreamStreamStreamId, s.name AS StreamStreamStreamStreamStreamStreamName, s.url AS StreamStreamStreamStreamStreamStreamUrl, s.total_view_count AS StreamStreamStreamStreamStreamStreamTotalViewCount, s.followers AS StreamStreamStreamStreamStreamStreamFollowers, s.createdAt AS StreamStreamStreamStreamStreamStreamCreatedAt, s.description AS StreamStreamStreamStreamStreamStreamDescription, s.id AS StreamStreamStreamStreamStreamStreamId"
524,"What is the total number of filings where the originator bank, beneficiary bank, and filer organization are all located in different countries?",MATCH (filing:Filing) WITH filing MATCH (filing)-[:ORIGINATOR]->(:Entity)-[:COUNTRY]->(originatorCountry:Country) MATCH (filing)-[:CONCERNS]->(:Entity)-[:COUNTRY]->(beneficiaryCountry:Country) MATCH (filing)-[:FILED]->(:Entity)-[:COUNTRY]->(filerCountry:Country) WHERE originatorCountry.code <> beneficiaryCountry.code AND originatorCountry.code <> filerCountry.code AND beneficiaryCountry.code <> filerCountry.code RETURN count(DISTINCT filing) AS totalFilings,MATCH (f:Filing) WHERE f.originator_bank_country <> f.beneficiary_bank_country AND f.originator_bank_country <> f.filer_org_name AND f.beneficiary_bank_country <> f.filer_org_name RETURN count(f)
992,"""who's your daddy""","MATCH (child:Person)-[:FOLLOWS]->(parent:Person) RETURN child.name AS ChildName, parent.name AS ParentName",MATCH (p:Person) WHERE p.name = 'your daddy' RETURN p
1794,List the entities associated with the 'Bahamas Leaks'.,MATCH (e:Entity) WHERE e.sourceID = 'Bahamas Leaks' RETURN e,"MATCH (e:Entity)-[:registered_address]->(a:Address)-[:same_id_as]->(:Other) WHERE a.sourceID = ""Bahamas Leaks"" RETURN e.name AS entity_name, a.address AS address, a.sourceID AS source_id"
504,Which entities are originators in the filings involving the highest amounts?,"MATCH (e:Entity)-[:ORIGINATOR]->(f:Filing) RETURN e, f.amount ORDER BY f.amount DESC LIMIT 1","MATCH (f:Filing) WITH f, f.amount AS filingAmount ORDER BY filingAmount DESC LIMIT 10 RETURN f.originator_bank_id AS originator_bank_id, f.originator_bank AS originator_bank, f.filer_org_name_id AS filer_o

In [238]:
pattern = r'["\']\d+["\']'
filtered_df = executable_df[executable_df['generated_cypher'].astype(str).str.contains(pattern, regex=True)][["question","cypher","generated_cypher"]]

In [239]:
filtered_df

,question,cypher,generated_cypher
11,How many users have an area other than 2.6 and a size less than 3?,MATCH (u:User) WHERE u.area <> 2.6 AND u.size < 3 RETURN count(u),MATCH (u:User) WHERE u.area <> '2.6' AND u.size < '3' RETURN count(u)
77,List the questions asked by the user with id 10251021.,"MATCH (u:User {id: 10251021})-[ :ASKED ]->(q:Question) RETURN q.title, q.id","MATCH (u:User {id: '10251021'})-[:ASKED]->(q:Question) RETURN q.title, q.link"
354,Find the names of all articles that mention organizations with a revenue greater than $1 billion.,MATCH (org:Organization)<-[:MENTIONS]-(article:Article) WHERE org.revenue > 1E9 RETURN article.title,MATCH (a:Article)-[:MENTIONS]->(o:Organization) WHERE o.revenue > '1000000000' RETURN a.title
439,List the first 3 countries where the beneficiary latitude is greater than 50 degrees.,MATCH (f:Filing) WHERE toFloat(f.beneficiary_lat) > 50 MATCH (f)-[:BENEFITS]->(e:Entity)-[:COUNTRY]->(c:Country) RETURN DISTINCT c.name LIMIT 3,"MATCH (f:Filing) WHERE f.beneficiary_lat > '50' WITH DISTINCT f.beneficiary_bank_country AS country_name, f.beneficiary_lat AS beneficiary_lat, f.beneficiary_lng AS beneficiary_lng RETURN country_name ORDER BY beneficiary_lat LIMIT 3"
554,"List the top 3 Filings where the amount is exactly $1,000,000.",MATCH (f:Filing) WHERE f.amount = 1000000 RETURN f ORDER BY f.amount DESC LIMIT 3,MATCH (f:Filing) WHERE f.amount = '1000000' RETURN f ORDER BY f.begin LIMIT 3
1182,List all movies that have been rated by user with ID 1.,MATCH (u:User {id: 1})-[:RATED]->(m:Movie) RETURN m.title AS MovieTitle,MATCH (u:User {id: '1'})-[:RATED]->(m:Movie) RETURN m.title
1207,What are the 3 latest movies rated by user with id 1?,"MATCH (u:User {id: 1})-[r:RATED]->(m:Movie) RETURN m.title, r.rating, r.timestamp ORDER BY r.timestamp DESC LIMIT 3","MATCH (u:User {id: '1'})-[:RATED]->(m:Movie) RETURN m.title, m.release_date ORDER BY m.release_date DESC LIMIT 3"
1239,What are the first 5 movies rated by user with ID 1?,MATCH (u:User {id: 1})-[:RATED]->(m:Movie) RETURN m LIMIT 5,"MATCH (u:User {id: '1'})-[:RATED]->(m:Movie) RETURN m.title, m.average_vote ORDER BY m.average_vote DESC LIMIT 5"
1247,List the top 3 people who have the most credits as a producer.,"MATCH (p:Person)-[r:CREW_FOR]->(m:Movie) WHERE r.job = ""Producer"" RETURN p.name, COUNT(r) AS producer_credits ORDER BY producer_credits DESC LIMIT 3","MATCH (p:Person)-[:CREW_FOR]->(m:Movie) WHERE m.id IN ['31357'] WITH p, count(m) AS producerCount ORDER BY producerCount DESC LIMIT 3 RETURN p.name AS producerName, producerCount"
1323,How many movies were released in the year 1990?,MATCH (m:Movie) WHERE date(m.release_date).year = 1990 RETURN count(m),MATCH (m:Movie) WHERE m.release_date = '1990' RETURN count(m)


In [109]:
def count_as(column):
    return column.str.count(r'\bas\b', flags=re.IGNORECASE)

excessive_df = executable_df[count_as(executable_df['generated_cypher']) > count_as(executable_df['cypher'])+1]

In [118]:
# excessive_df[["question","cypher","generated_cypher"]]

In [111]:
len(excessive_df)

130

In [123]:
df.iloc[14]

Unnamed: 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           1229
question                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [219]:
def count_distinct_kw(column):
    return column.str.count(r'\bdistinct\b', flags=re.IGNORECASE)

df_distinctkw = executable_df[count_distinct_kw(executable_df['generated_cypher']) < count_distinct_kw(executable_df['cypher'])]

In [220]:
df_distinctkw[df_distinctkw["bleu_score"] < 0.4][["question","cypher","generated_cypher"]]
#df_distinctkw

,question,cypher,generated_cypher
67,What is the total number of questions asked by users with a reputation greater than the average reputation of all users?,MATCH (u:User) WITH avg(u.reputation) AS averageReputation MATCH (u:User)-[:ASKED]->(q:Question) WHERE u.reputation > averageReputation RETURN count(DISTINCT q) AS questionCount,"MATCH (u:User)-[:ASKED]->(q:Question) WITH u, q RETURN u.reputation AS user_reputation, q.title AS question_title, q.link AS question_link, q.createdAt AS question_created_at, q.upVotes AS question_upvotes, q.downVotes AS question_downvotes, q.score AS question_score UNION ALL MATCH (u:User)-[:ASKED]->(q:Question) WITH u, q RETURN u.reputation AS user_reputation, q.title AS question_title, q.link AS question_link, q.createdAt AS question_created_at, q.upVotes AS question_upvotes, q.downVotes AS question_downvotes, q.score AS question_score ORDER BY q.createdAt DESC"
176,Which organizations have a revenue decrease mentioned in any articles?,MATCH (a:Article)-[:MENTIONS]->(o:Organization) WHERE a.sentiment < 0 AND exists { (a)-[:HAS_CHUNK]->(c:Chunk) WHERE c.text CONTAINS 'revenue decrease' OR c.text CONTAINS 'revenue declined' } RETURN DISTINCT o.name AS organization,MATCH (o:Organization)-[:MENTIONS]->(a:Article) WHERE a.summary CONTAINS 'revenue decrease' RETURN o.name
421,Which organizations have been mentioned in articles about corporate social responsibility?,"MATCH (a:Article)-[:HAS_CHUNK]->(c:Chunk) WHERE c.text CONTAINS ""corporate social responsibility"" MATCH (a)-[:MENTIONS]->(o:Organization) RETURN DISTINCT o.name",MATCH (a:Article)-[:MENTIONS]->(o:Organization) WHERE a.summary CONTAINS 'corporate social responsibility' RETURN o.name
524,"What is the total number of filings where the originator bank, beneficiary bank, and filer organization are all located in different countries?",MATCH (filing:Filing) WITH filing MATCH (filing)-[:ORIGINATOR]->(:Entity)-[:COUNTRY]->(originatorCountry:Country) MATCH (filing)-[:CONCERNS]->(:Entity)-[:COUNTRY]->(beneficiaryCountry:Country) MATCH (filing)-[:FILED]->(:Entity)-[:COUNTRY]->(filerCountry:Country) WHERE originatorCountry.code <> beneficiaryCountry.code AND originatorCountry.code <> filerCountry.code AND beneficiaryCountry.code <> filerCountry.code RETURN count(DISTINCT filing) AS totalFilings,MATCH (f:Filing) WHERE f.originator_bank_country <> f.beneficiary_bank_country AND f.originator_bank_country <> f.filer_org_name AND f.beneficiary_bank_country <> f.filer_org_name RETURN count(f)
570,List the top 3 countries by the number of entities that have filed more than 30 filings.,"MATCH (e:Entity)-[:FILED]->(f:Filing) WITH e, count(f) AS filingsCount WHERE filingsCount > 30 MATCH (e)-[:COUNTRY]->(c:Country) RETURN c.name, count(DISTINCT e) AS entityCount ORDER BY entityCount DESC LIMIT 3","MATCH (e:Entity)-[:FILED]->(f:Filing) WITH e, count(f) AS filing_count WHERE filing_count > 30 WITH e, filing_count, e.name AS entity_name, e.location AS entity_location, e.country AS entity_country ORDER BY filing_count DESC LIMIT 3 RETURN e.country AS country, filing_count AS filing_count"
584,"Which characters interacted with both ""Catelyn-Stark"" and ""Cersei-Lannister""?","MATCH (catelyn:Character {name: ""Catelyn-Stark""})-[:INTERACTS]->(catelyn_interaction) WITH catelyn, count(DISTINCT catelyn_interaction) AS catelyn_interactions MATCH (cersei:Character {name: ""Cersei-Lannister""})-[:INTERACTS]->(cersei_interaction) WITH catelyn, cersei, catelyn_interactions, count(DISTINCT cersei_interaction) AS cersei_interactions MATCH (catelyn)-[:INTERACTS]->(common_interaction)<-[:INTERACTS]-(cersei) RETURN DISTINCT common_interaction.name","MATCH (c1:Character {name: ""Catelyn-Stark""})-[:INTERACTS45]->(c2:Character), (c3:Character {name: ""Cersei-Lannister""})-[:INTERACTS45]->(c2) RETURN DISTINCT c2.name"
611,Find all characters that have interacted in 'book 2' and have a 'louvain' value greater than 3.,MATCH (c:Character)-[:INTERACTS2]->(c2:Character) WHERE c.louv